In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## import libraries

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report , accuracy_score , confusion_matrix
from imblearn.under_sampling import RandomUnderSampler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from scipy.stats import skew 
import scipy.stats as stats
from scipy.stats import kurtosis

## concat data

In [ ]:
dfs = []
dir = '/kaggle/input/signal/10g'
for filename in os.listdir(dir):
    if filename.endswith('.csv'):
        file_path = os.path.join(dir, filename)
        dfs.append(pd.read_csv(file_path, header=None))

df_10g = pd.concat(dfs, ignore_index=True)

In [ ]:
df_10g.shape

In [ ]:
df_10g['type'] = 2

In [ ]:
dfs = []
dir = '/kaggle/input/signal/6g'
for filename in os.listdir(dir):
    if filename.endswith('.csv'):
        file_path = os.path.join(dir, filename)
        dfs.append(pd.read_csv(file_path, header=None))

df_6g = pd.concat(dfs, ignore_index=True)

In [ ]:
df_6g.shape

In [ ]:
df_6g['type'] = 1

In [ ]:
dfs = []
dir = '/kaggle/input/signal/normal'
for filename in os.listdir(dir):
    if filename.endswith('.csv'):
        file_path = os.path.join(dir, filename)
        dfs.append(pd.read_csv(file_path, header=None))

df_normal = pd.concat(dfs, ignore_index=True)

In [ ]:
df_normal.shape

In [ ]:
df_normal['type'] = 0

In [ ]:
new_class = pd.concat([df_normal , df_6g , df_10g])

In [ ]:
x=new_class.drop(['type'],axis=1)
y=new_class['type']

In [ ]:
rus = RandomUnderSampler(sampling_strategy='auto')
x, y = rus.fit_resample(x, y)

In [ ]:
y.value_counts()


In [ ]:
new_class = pd.concat([x, y], axis=1)


In [ ]:
new_class['type'].value_counts()

In [ ]:
grouped = new_class.groupby('type')

# Create a dictionary to hold separate DataFrames for each class
class_dfs = {name: group for name, group in grouped}

# Accessing individual DataFrames
df_normal = class_dfs[0]
df_6g = class_dfs[1]
df_10g = class_dfs[2]

In [ ]:
df_normal.head()

In [ ]:
df_normal.shape

## windowing

In [ ]:
def create_windowed_dataframe(data, window_size):
    windowed_arrays = []
    df_info = pd.DataFrame()
    
    for i in range(0, len(data), window_size):
        df_info = data.iloc[i:i + window_size]
        for j in range(0,7):
            array = df_info.iloc[:, j:j+1].values
            windowed_arrays.append(array)
            
    return windowed_arrays
    
   

In [ ]:
window_size = 100
df_t_10g = create_windowed_dataframe(df_10g, window_size)
means = [np.mean(arr) for arr in df_t_10g]
df_w_10g = pd.DataFrame({
        'mean': means
    })




df_w_10g.head()    
    

In [ ]:
df_w_10g.shape

In [ ]:
def median(new_dataframe,data):
    new_dataframe['meadian'] = [np.median(arr) for arr in data]


In [ ]:
def std(new_dataframe,data):
    new_dataframe['std'] = [np.std(arr) for arr in data]


In [ ]:
def maximume(new_dataframe,data):
    new_dataframe['max'] = [np.max(arr) for arr in data]

In [ ]:
def minimume(new_dataframe,data):
    new_dataframe['min'] = [np.min(arr) for arr in data]

In [ ]:
def q1(new_dataframe,data):
    new_dataframe['q1'] = [np.quantile(arr,0.25) for arr in data]
def q3(new_dataframe,data):
    new_dataframe['q3'] = [np.quantile(arr,0.75) for arr in data]

In [ ]:
def skewness(new_dataframe,data):
    skewness = [skew(arr.flatten()) for arr in data]
    new_dataframe['skewness'] = pd.to_numeric(skewness)

In [ ]:
def calculate_rms(new_dataframe,data):
    new_dataframe['RMS'] =  [np.sqrt(np.mean(arr**2)) for arr in data]
 

In [ ]:
 def var(new_dataframe,data):
    new_dataframe['var'] = [np.var(arr) for arr in data] 

In [ ]:
skewness(df_w_10g,df_t_10g)

In [ ]:
std(df_w_10g,df_t_10g)

In [ ]:
maximume(df_w_10g,df_t_10g)
minimume(df_w_10g,df_t_10g)

In [ ]:
median(df_w_10g,df_t_10g)


In [ ]:
q1(df_w_10g,df_t_10g)

In [ ]:
 q3(df_w_10g,df_t_10g)

In [ ]:
calculate_rms(df_w_10g, df_t_10g)


In [ ]:
var(df_w_10g, df_t_10g)

In [ ]:
df_w_10g['type'] = 2
df_w_10g.head()

## 6g

In [ ]:
window_size = 100
df_t_6g = create_windowed_dataframe(df_6g, window_size)
means = [np.mean(arr) for arr in df_t_6g]
df_w_6g = pd.DataFrame({
        'mean': means
    })



df_w_6g.head()    

In [ ]:
df_w_6g.shape

In [ ]:
skewness(df_w_6g,df_t_6g)

In [ ]:
maximume(df_w_6g,df_t_6g)
minimume(df_w_6g,df_t_6g)

In [ ]:
std(df_w_6g,df_t_6g)

In [ ]:
median(df_w_6g,df_t_6g)

In [ ]:
q1(df_w_6g,df_t_6g) 
q3(df_w_6g,df_t_6g)

In [ ]:
calculate_rms(df_w_6g,df_t_6g)

In [ ]:
var(df_w_6g,df_t_6g)

In [ ]:
df_w_6g['type'] = 1
df_w_6g.head()

## normal

In [ ]:
window_size = 100
normal = create_windowed_dataframe(df_normal, window_size)
means = [np.mean(arr) for arr in normal]
df_w_normal = pd.DataFrame({
        'mean': means
    })



df_w_normal.head()    

In [ ]:
skewness(df_w_normal,normal)

In [ ]:
maximume(df_w_normal,normal)
minimume(df_w_normal,normal)

In [ ]:
std(df_w_normal,normal)

In [ ]:
median(df_w_normal,normal)

In [ ]:
q1(df_w_normal,normal)
q3(df_w_normal,normal)

In [ ]:
calculate_rms(df_w_normal,normal)

In [ ]:
var(df_w_normal,normal)

In [ ]:
df_w_normal['type'] = 0
df_w_normal.head()

In [ ]:
new_df = pd.concat([df_w_normal , df_w_6g , df_w_10g])

In [ ]:
sns.heatmap(new_df.corr(numeric_only = True))

In [ ]:
X = new_df.iloc[:,0:10]
y = new_df.iloc[:,10]

In [ ]:
X_train , X_test , y_train , y_test = train_test_split(X,y,random_state=11,test_size=0.2)

In [ ]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)

In [ ]:
X_test = scaler.fit_transform(X_test)

In [ ]:
rf_model = RandomForestClassifier()
rf_model.fit(X_train, y_train)


In [ ]:
importances = rf_model.feature_importances_

In [ ]:
feature_names = ['mean','skewness','std','max','min','meadian','q1','q3','RMS','var']
X_train_df = pd.DataFrame(X_train, columns=feature_names)

In [ ]:
plt.figure(figsize=(12, 6))
plt.bar(range(X_train_df.shape[1]), importances)
plt.xticks(range(X_train_df.shape[1]), X_train_df.columns, rotation=90)
plt.ylabel('Feature Importance Scores')
plt.xlabel('Features')
plt.show()

## KNN 

In [ ]:
KNN = KNeighborsClassifier()

In [ ]:
KNN.fit(X_train, y_train)

In [ ]:
y_pred = KNN.predict(X_test)

In [ ]:
KNN.score(X_test,y_test)

In [ ]:
print("Accuracy:", accuracy_score(y_test, y_pred))

##  drropping less important feature

In [ ]:
new_df1 = new_df.drop(['skewness'] , axis=1)

In [ ]:
new_df1.head()

In [ ]:
X = new_df1.iloc[:,0:9]
y = new_df1.iloc[:,9]

In [ ]:
X_train , X_test , y_train , y_test = train_test_split(X,y,random_state=11,test_size=0.2)

In [ ]:
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [ ]:
KNN.fit(X_train, y_train)

In [ ]:
pred = KNN.predict(X_test)

In [ ]:
KNN.score(X_test,y_test)

In [ ]:
print("Accuracy:", accuracy_score(y_test, pred))

## hyper parameter tunning

### KNN

In [ ]:
param_grid = {
    'n_neighbors': [5, 8, 10, 12],  
    'metric': ['euclidean', 'manhattan']  
}

In [ ]:
gs = GridSearchCV(KNN,param_grid,cv=10)

In [ ]:
X = new_df.iloc[:,0:10]
y = new_df.iloc[:,10]

In [ ]:
X_train , X_test , y_train , y_test = train_test_split(X,y,random_state=11,test_size=0.2)

In [ ]:
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [ ]:
gs.fit(X_train, y_train)

In [ ]:
best_k = gs.best_params_['n_neighbors']
best_metric = gs.best_params_['metric']

print(f"Best k: {best_k}, Best metric: {best_metric}")

In [ ]:
accuracy = gs.score(X_test, y_test)
print(f"Test set accuracy: {accuracy}")

### Random Forest

In [ ]:
RF = RandomForestClassifier()

In [ ]:
param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
}


In [ ]:
gs = GridSearchCV(estimator = RF, param_grid = param_grid, cv = 3, n_jobs = -1, verbose = 2)

In [ ]:
gs.fit(X_train, y_train)

In [ ]:
print(gs.score(X_test,y_test))

### DecisionTree 

In [ ]:
from sklearn.tree import DecisionTreeRegressor

param_grid = {
    'max_depth': [10, 20, 30, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
dtree_reg = DecisionTreeRegressor()
gs = GridSearchCV(estimator=dtree_reg, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)

In [ ]:
gs.fit(X_train, y_train)

In [ ]:
best_params = grid_search.best_params_
print(gs.score(X_test,y_test))

### logestic reg

In [ ]:
lg = LogisticRegression()
param_grid = {
    'solvers' : ['newton-cg', 'lbfgs', 'liblinear'],
    'c_values' : [100, 10, 1.0, 0.1, 0.01]
}


In [ ]:
gs = GridSearchCV(estimator=lg,param_grid=param_grid)
grid_result = gs.fit(X_train, y_train)

In [ ]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

## nural network

In [ ]:
import tensorflow as tf
from tensorflow.keras import models, layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM


In [ ]:
regressor = Sequential()
# First LSTM layer 
regressor.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1],1)))
# Second LSTM layer
regressor.add(LSTM(units=50, return_sequences=True))
# Third LSTM layer
regressor.add(LSTM(units=50, return_sequences=True))
# Fourth LSTM layer
regressor.add(LSTM(units=50))
# The output layer
regressor.add(Dense(units=1))

regressor.compile(optimizer='rmsprop',loss='mean_squared_error')
regressor.fit(X_train,y_train,epochs=20,batch_size=32)